In [11]:
# installing required magic packages
!pip install sqlalchemy
!pip install ipython-sql
!pip install pymysql

In [13]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# connect to the sqlite database
%load_ext sql

engine = create_engine("sqlite:///bonga_ecommerce.db")
%config SQL.conn_name = 'engine'

conn = sqlite3.connect("bonga_ecommerce.db")
cursor = conn.cursor()

%sql sqlite:///bonga_ecommerce.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Create Tables

In [15]:
# DATA DEFINITION LANGUAGE (DDL)
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id integer primary key,
    name text not null,
    price decimal not null,
    category text not null
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id integer primary key,
    name text not null,
    email uuid not null
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id integer primary key,
    customer_id integer not null,
    order_date date not null,
    foreign key (customer_id) references customers(customer_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    order_item_id integer primary key,
    order_id integer not null,
    product_id integer not null,
    quantity integer not null,
    foreign key (order_id) references orders(order_id),
    foreign key (product_id) references product(product_id)
)
''')


Converting the python cell to sql

In [ ]:
!pip install --upgrade prettytable

from prettytable import DEFAULT
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
%%sql 
SELECT name FROM sqlite_master WHERE type in ('table', 'view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///bonga_ecommerce.db
Done.


name
customers
order_items
order_itemss
orders
products


Loading data into Database tables

In [18]:
# Function to load data into the database tables
def load_data_to_table(csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists = 'append', index=False)

In [ ]:
load_data_to_table('products.csv', 'products')
load_data_to_table('customers.csv', 'customers')
load_data_to_table('orders.csv', 'orders')
load_data_to_table('order_items.csv', 'order_items')

In [ ]:
# A NEW PRODUCT ADDED
%sql INSERT INTO products (name, price, category) VALUES ('Golden book', 345.88, 'Books')

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

In [20]:
%sql select * from products LIMIT(5)

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Golden book,345.88,Books
2,Golden book,345.88,Books


In [ ]:
# ALSO
# cursor.execute("INSERT INTO products (name, price, category) VALUES (?,?,?)", ('lappy', 895.88, 'Electronics'))
cursor.execute("INSERT INTO products (name, price, category) VALUES (?,?,?)", ('laptop', 345.18, 'Electronics'))
conn.commit()

 * sqlite:///bonga_ecommerce.db
Done.


product_id,name,price,category
1,Golden book,345.88,Books
2,Golden book,345.88,Books
3,laptop,345.18,Electronics


In [23]:
# UPDATE
%sql UPDATE customers SET email = 'herday@gmail.com' WHERE name = 'Customer1'

 * sqlite:///bonga_ecommerce.db
1 rows affected.


[]

### Tasks

In [33]:
%%sql
SELECT category, 
       SUM(price * quantity) AS total_sales
FROM order_items AS op, 
     products AS p
WHERE op.product_id = p.product_id
GROUP BY category;

 * sqlite:///bonga_ecommerce.db
Done.


category,total_sales


Total Sales of Product Per Category

In [ ]:
%%sql
SELECT products.category, SUM( order_items.quantity) AS total_price FROM order_items JOIN products ON order_items.product_id = products.product_id GROUP BY products.category


 * sqlite:///bonga_ecommerce.db
Done.


category,total_price
Books,3272
Electronics,3025
Fashion,3057
Home & Kitchen,2558
Toys,3065


Highest selling price

In [ ]:
%%sql
SELECT product_id, 
sum(quantity) as 'Total_quantity'
from order_items
group by product_id
order by Total_quantity DESC
limit(10)

 * sqlite:///bonga_ecommerce.db
Done.


product_id,Total_quantity
79,41
439,40
130,38
921,33
128,33
106,33
939,32
919,32
798,31
551,31


Highest selling product

In [ ]:
%%sql
select product_id, sum (quantity) as total_quantity
from order_items
group by product_id
order by total_quantity desc
limit 1;

 * sqlite:///bonga_ecommerce.db
Done.


product_id,total_quantity
79,41


Total number of Products that are Electronics

In [ ]:
%%sql
select count(*) as 'Number of Electronics Product'
from products
where category is 'Electronics'

 * sqlite:///bonga_ecommerce.db
Done.


Number of Electronics Product
193



Categories with more than 10 sales

In [ ]:
%%sql

SELECT category, count(od.order_id) as 'orders_count'
FROM orders as od, products as p, order_items as op
WHERE od.order_id = op.order_id and 
    op.product_id = p.product_id
GROUP BY category 
HAVING orders_count > 10 ;

 * sqlite:///bonga_ecommerce.db
Done.


category,orders_count
Books,1302
Electronics,1181
Fashion,1247
Home & Kitchen,1019
Toys,1251


Average product price per categories ordered by price

In [ ]:
%%sql
SELECT category, Avg(price) as 'Average_Price'
FROM products
GROUP BY category
ORDER BY Average_Price DESC

 * sqlite:///bonga_ecommerce.db
Done.


category,Average_Price
Fashion,261.46653846153845
Electronics,257.07169230769233
Home & Kitchen,253.98537572254335
Books,246.61542056074765
Toys,235.41691588785048


Product name and Order_date

In [ ]:
%%sql

SELECT p.name, od.order_date
FROM products as p
INNER JOIN order_items as oi ON p.product_id = oi.product_id
INNER JOIN orders as od ON oi.order_id = od.order_id
limit(5)

 * sqlite:///bonga_ecommerce.db
Done.


name,order_date
Product 34,2014-09-04 17:49:21
Product 653,2014-09-04 17:49:21
Product 427,2014-09-04 17:49:21
Product 830,2011-07-19 03:46:15
Product 886,2011-07-19 03:46:15
